In [1]:
### Cargamos las dependencias

import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# Definimos el nombre del archivo CSV y las columnas que vamos mantener

# Por cuestiones de almacenamiento no vamos a cargar este ya que pesa 550 Mb
archivo_csv = './archivos_csv/precios-historicos.csv'
columnas = ['idempresa', 'empresa', 'direccion', 'localidad', 'provincia', 'idproducto', 'producto', 'idtipohorario', 'precio', 'fecha_vigencia', 'idempresabandera', 'empresabandera', 'latitud', 'longitud']

# Definimos el tamaño de los lotes
batch_size = 100000

# Carga del archivo CSV en lotes
chunks = pd.read_csv(archivo_csv, usecols=columnas, chunksize=batch_size)

# Creamos una lista para almacenar los lotes que ya han sido procesados
lotes_procesados = []

# Iteramos sobre cada lote
for chunk in chunks:
    # Eliminamos las filas donde 'id_horario' es igual a 3
    chunk = chunk[chunk['idtipohorario'] != 3]
    
    # Agregamos el lote procesado a la lista
    lotes_procesados.append(chunk)

# Concatenamos todos los lotes procesados en un solo DataFrame
df = pd.concat(lotes_procesados, ignore_index=True)

print("✅ Dataset cargado correctamente! ")
print()

# Eliminamos la columna de 'idtipohorario' ya que a la misma no le daremos uso
df.drop('idtipohorario', axis=1, inplace=True)

# Le damos un nuevo formato a la fecha, pasando del formato '01/04/2024 10:11' a '01/04/2024'
df["fecha_vigencia"] = df["fecha_vigencia"].apply(lambda x: datetime.strptime(x, "%d/%m/%Y %H:%M").date().strftime("%d/%m/%Y"))

# Eliminamos los registros con años fuera del rango de 2001 a 2024
# Definimos las variables de años
anio_min = 2001
anio_max = 2024

# Recorremos el dataset original
for fila in df.itertuples():
  # Convertimos la fecha en un tipo de dato a analizar
  fecha_reg = datetime.strptime(fila.fecha_vigencia, "%d/%m/%Y")
  # Extraemos el año de la fecha
  anio = fecha_reg.year

  # Filtramos las filas por año
  if anio_min <= anio <= anio_max:
    continue
  else:
    # Eliminamos los registros con fechas fuera del rango
    df.drop(index=fila.Index, inplace=True)

# Imprimimos un mensaje al finalizar
print("✅ Los registros fuera del rango de fechas se han eliminado correctamente!")
print()

# Renombramos las Columnas
df.rename({
    'idempresa': 'id_empresa', 
    'empresa': 'nombre_empresa', 
    'fecha_vigencia': 'fecha_vigencia',
    'localidad': 'localidad',
    'provincia': 'provincia',
    'idproducto': 'id_producto',
    'producto': 'nombre_producto',
    'precio': 'precio',
    'idempresabandera': 'id_empresa_bandera',
    'empresabandera': 'nombre_empresa_bandera',
    'latitud': 'latitud',
    'longitud': 'longitud',
    }, axis=1, inplace=True)

# Asignamos las regiones a cada empresa 
for fila in df.itertuples():
  # Obtener la provincia
  provincia = fila.provincia
  
  # Asignar la región según la provincia
  if provincia in ("CORDOBA", "ENTRE RIOS", "SANTA FE"):
    df.loc[fila.Index, "region"] = "Centro"
  elif provincia in ("MENDOZA", "SAN JUAN", "SAN LUIS", "TUCUMAN"):
    df.loc[fila.Index, "region"] = "Cuyo"
  elif provincia in ("CHACO", "CORRIENTES", "FORMOSA", "MISIONES"):
    df.loc[fila.Index, "region"] = "Noreste"
  elif provincia in ("CATAMARCA", "JUJUY", "LA RIOJA", "SALTA", "SANTIAGO DEL ESTERO"):
    df.loc[fila.Index, "region"] = "Noroeste"
  elif provincia in ("BUENOS AIRES", "CAPITAL FEDERAL", "LA PAMPA"):
    df.loc[fila.Index, "region"] = "Pampeana"
  elif provincia in ("CHUBUT", "NEUQUEN", "RIO NEGRO", "SANTA CRUZ", "TIERRA DEL FUEGO"):
    df.loc[fila.Index, "region"] = "Patagonia"

print("✅ Proceso terminado!")
print()

df


✅ Dataset cargado correctamente! 

✅ Los registros fuera del rango de fechas se han eliminado correctamente!

✅ Proceso terminado!



,id_empresa,nombre_empresa,direccion,localidad,provincia,id_producto,nombre_producto,precio,fecha_vigencia,id_empresa_bandera,nombre_empresa_bandera,latitud,longitud,region
0,1376,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,19,Gas Oil Grado 2,985.00,01/04/2024,28,PUMA,-34.658476,-58.529443,Pampeana
1,1376,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,19,Gas Oil Grado 2,172.50,01/10/2022,28,PUMA,-34.658476,-58.529443,Pampeana
2,1376,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,19,Gas Oil Grado 2,925.00,02/03/2024,28,PUMA,-34.658476,-58.529443,Pampeana
3,1376,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,19,Gas Oil Grado 2,91.20,03/06/2021,28,PUMA,-34.658476,-58.529443,Pampeana
4,1376,10 DE SETIEMBRE S.A.,Av. Mosconi 299,LOMAS DEL MIRADOR,BUENOS AIRES,19,Gas Oil Grado 2,831.00,04/01/2024,28,PUMA,-34.658476,-58.529443,Pampeana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1297236,4097,ZUSICO S.A.,TALCAHUANO 1198,MAR DEL PLATA,BUENOS AIRES,2,Nafta (súper) entre 92 y 95 Ron,72.81,28/12/2020,28,PUMA,-38.031169,-57.575518,Pampeana
1297237,4097,ZUSICO S.A.,TALCAHUANO 1198,MAR DEL PLATA,BUENOS AIRES,2,Nafta (súper) entre 92 y 95 Ron,41.39,29/09/2018,28,PUMA,-38.031169,-57.575518,Pampeana
1297238,4097,ZUSICO S.A.,TALCAHUANO 1198,MAR DEL PLATA,BUENOS AIRES,2,Nafta (súper) entre 92 y 95 Ron,46.37,30/03/2019,28,PUMA,-38.031169,-57.575518,Pampeana
1297239,4097,ZUSICO S.A.,TALCAHUANO 1198,MAR DEL PLATA,BUENOS AIRES,2,Nafta (súper) entre 92 y 95 Ron,46.21,30/04/2019,28,PUMA,-38.031169,-57.575518,Pampeana


In [3]:
# Vamos a eliminar aquellos registros que no tengan datos de latitud y longitud
for fila in df.itertuples():
  # Revisar si la latitud está vacía
  if pd.isnull(fila.latitud):
    # Buscar coincidencias en la columna "empresa"
    coincidencias = df.loc[df["nombre_empresa"] == fila.nombre_empresa]
    # Si hay coincidencias
    if len(coincidencias) > 1:
      # Buscar la primera coincidencia con latitud no nula
      for coincidencia in coincidencias.itertuples():
        if not pd.isnull(coincidencia.latitud):
          # Reemplazar valores con la coincidencia
          df.loc[fila.Index, "longitud"] = coincidencia.longitud
          df.loc[fila.Index, "latitud"] = coincidencia.latitud
          break

print("✅ Se han reemplazado los registros que tenian coincidencias!")
print()

for fila in df.itertuples():
  # Si la fila no tiene latitud ni longitud
  if pd.isnull(fila.latitud) and pd.isnull(fila.longitud):
    # Eliminar la fila
    df.drop(fila.Index, inplace=True)
    
print("✅ Se han eliminado los registros sin latitud ni longitud satisfactoriamente!")
print()

print("🚩 Prueba de elementos nulos en el dataset:")
print()

df.isnull().sum()

✅ Se han reemplazado los registros que tenian coincidencias!

✅ Se han eliminado los registros sin latitud ni longitud satisfactoriamente!

🚩 Prueba de elementos nulos en el dataset:



id_empresa                0
nombre_empresa            0
direccion                 0
localidad                 0
provincia                 0
id_producto               0
nombre_producto           0
precio                    0
fecha_vigencia            0
id_empresa_bandera        0
nombre_empresa_bandera    0
latitud                   0
longitud                  0
region                    0
dtype: int64

In [11]:
# Creacion Dataset para empresas

# Extraemos solo las columnas que nos hacen falta
df_copy = df[["id_empresa", "nombre_empresa", "id_empresa_bandera", "direccion", "localidad", "provincia", "region", "latitud", "longitud"]]

# Eliminamos todos los registros que tengan un id duplicado
df_empresas = df_copy.drop_duplicates(subset=["id_empresa"])

# Vamos a darles formato object a la columna de latitud y longitud
df_empresas["latitud"].astype("object")
df_empresas["longitud"].astype("object")

# Ordenamos el dataset
df_empresas_sorted = df_empresas.sort_values(by="id_empresa")

# Lo actualizamos
df_empresas = df_empresas_sorted

# Mostramos el resultado
df_empresas



,id_empresa,nombre_empresa,id_empresa_bandera,direccion,localidad,provincia,region,latitud,longitud
818722,102,M.P.C.I. S.A.,20,RUTA 3 KM. 298,AZUL,BUENOS AIRES,Pampeana,-36.788170,-59.834700
180602,104,CALCAGNO RICARDO OMAR Y CALCAGNO RICARDO JAVIE...,4,AV. 25 DE MAYO 280,RESISTENCIA,CHACO,Noreste,-27.448803,-58.988820
705886,105,LAS SIERRAS S.R.L,1,RUTA 36 Y ACCESO 005,RIO CUARTO,CORDOBA,Centro,-33.081140,-64.339980
751915,107,MACHAGAI COMBUSTIBLES SRL,2,M.MORENO Y 25 DE MAYO,MACHAGAY,CHACO,Noreste,-26.924700,-60.046930
629305,108,JACQUELIN O Y CARIGNANO J S DE HECHO,2,GENERAL LOPEZ 433,WHEELWRIGHT,SANTA FE,Centro,-33.791756,-61.206679
...,...,...,...,...,...,...,...,...,...
556772,10964,G.N.C. CALAMUCHITA S.R.L.,6,HIPOLITO YRIGOYEN 599,SANTA ROSA DE CALAMUCHITA,CORDOBA,Centro,-32.071690,-64.528370
1181650,10965,SERVICIOS TARALLI SA,28,RUTA PROVINCIAL 21 Y 24 DE SEPTIEMBRE 2,BANDERA,SANTIAGO DEL ESTERO,Noroeste,-28.889600,-62.272870
706409,10967,LAS VEGAS SA,29,RUTA 6 KM 57,FRANCK,SANTA FE,Centro,-31.578650,-60.936580
443131,10968,ESTACION DEHEZA S.R.L.,1,AV. PTE RAUL ALFONSIN 2580/82,LANUS,BUENOS AIRES,Pampeana,-34.709900,-58.366622


In [5]:
# Creacion Dataset para productos

# Extraemos solo las columnas que nos hacen falta
df_copy = df[["id_producto", "nombre_producto"]]

# Eliminamos todos los registros que tengan un id duplicado
df_productos = df_copy.drop_duplicates(subset=["id_producto"])

# Ordenamos el dataset
df_productos_sorted = df_productos.sort_values(by="id_producto")

# Lo actualizamos
df_productos = df_productos_sorted

# Mostramos el resultado
df_productos

,id_producto,nombre_producto
32,2,Nafta (súper) entre 92 y 95 Ron
24,3,Nafta (premium) de más de 95 Ron
20,6,GNC
0,19,Gas Oil Grado 2
12,21,Gas Oil Grado 3


In [6]:
# Creacion Dataset para empresas de bandera

# Extraemos solo las columnas que nos hacen falta
df_copy = df[["id_empresa_bandera", "nombre_empresa_bandera"]]

# Eliminamos todos los registros que tengan un id duplicado
df_empresas_bandera = df_copy.drop_duplicates(subset=["id_empresa_bandera"])

# Ordenamos el dataset
df_empresas_bandera_sorted = df_empresas_bandera.sort_values(by="id_empresa_bandera")

# Lo actualizamos
df_empresas_bandera = df_empresas_bandera_sorted

# Mostramos el resultado
df_empresas_bandera

,id_empresa_bandera,nombre_empresa_bandera
15675,-1,SIN EMPRESA BANDERA
1341,1,BLANCA
433,2,YPF
18354,3,ESSO PETROLERA ARGENTINA S.R.L
341,4,SHELL C.A.P.S.A.
14651,6,DAPSA S.A.
78996,7,SOL PETROLEO
12061,8,REFINOR
565080,17,ASPRO
60170,20,PETROBRAS


In [7]:
# Ahora que tenemos modularizado nuestro Dataset vamos a eliminar las columnas que no necesitamos
df_final = df.copy()

df_final.drop(columns=["nombre_empresa", "direccion", "localidad", "provincia", "region", "latitud", "longitud", "nombre_producto", "id_empresa_bandera", "nombre_empresa_bandera"], inplace=True)

df_final = df_final.reindex(columns=["fecha_vigencia", "id_producto", "precio", "id_empresa"])

df_final


,fecha_vigencia,id_producto,precio,id_empresa
0,01/04/2024,19,985.00,1376
1,01/10/2022,19,172.50,1376
2,02/03/2024,19,925.00,1376
3,03/06/2021,19,91.20,1376
4,04/01/2024,19,831.00,1376
...,...,...,...,...
1297236,28/12/2020,2,72.81,4097
1297237,29/09/2018,2,41.39,4097
1297238,30/03/2019,2,46.37,4097
1297239,30/04/2019,2,46.21,4097


In [12]:
# Exportamos los diferentes datasets obtenidos

# Dataset Base
df_final.to_csv("./archivos_csv/precios-historicos-argentina-2001-2024.csv", index=False)

# Dataset Productos
df_productos.to_csv("./archivos_csv/listado-de-productos.csv", index=False)

# Dataset Empresas
df_empresas.to_csv("./archivos_csv/listado-de-empresas.csv", index=False)

# Dataset Empresas de Bandera
df_empresas_bandera.to_csv("./archivos_csv/listado-empresas-bandera.csv", index=False)
